In [102]:
import wget
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
output_dir = './'

output_file = 'NYTData.csv'


url = "https://pastebin.com/raw/YEHWZaZD"#

wget.download(url, out = os.path.join(output_dir,output_file))

'./NYTData (1).csv'

In [103]:
df = pd.read_csv('./NYTData.csv')


In [104]:
#How often is 3rd party change negative? 
print(df.where(df['3rd party change']<0).shape[0])

334


In [105]:
#Woah, why's that? 
df[df['3rd party change'] < 0].head()

,timestamp,total votes,trump %,biden %,3rd party %,trump votes,biden votes,3rd party votes,3rd party change,3rd party % change,trump change,trump % change,biden change,biden % change,total change
3,2020-11-04T02:44:00Z,656769,0.305,0.676,0.019,200314.545,443975.844,12478.611,-645.009,-0.049149,835.521,0.004189,397.488,0.000896,588
15,2020-11-04T03:33:07Z,1243486,0.372,0.608,0.020,462576.792,756039.488,24869.720,-1115.764,-0.042938,4737.312,0.010347,2460.452,0.003265,6082
17,2020-11-04T03:42:54Z,1300222,0.378,0.602,0.020,491483.916,782733.644,26004.440,-230.650,-0.008792,26748.036,0.057555,24414.614,0.032196,50932
23,2020-11-04T03:56:00Z,1420621,0.393,0.587,0.020,558304.053,833904.527,28412.420,-1387.021,-0.046545,2047.821,0.003681,939.200,0.001128,1600
25,2020-11-04T03:56:42Z,1425046,0.394,0.586,0.020,561468.124,835076.956,28500.920,-1404.256,-0.046957,1814.116,0.003241,580.140,0.000695,990


In [116]:
#Woah, why's that? 
df[df['trump change'] < 0]

,timestamp,total votes,trump %,biden %,3rd party %,trump votes,biden votes,3rd party votes,3rd party change,3rd party % change,trump change,trump % change,biden change,biden % change,total change
26,2020-11-04T03:58:32Z,1425651,0.393,0.586,0.021,560280.843,835431.486,29938.671,1437.751,5.044578e-02,-1187.281,-2.114601e-03,354.530,4.245477e-04,605
34,2020-11-04T04:16:57Z,1583386,0.406,0.573,0.021,642854.716,907280.178,33251.106,1583.606,5.000729e-02,-1578.909,-2.450072e-03,6.303,6.947186e-06,11
51,2020-11-04T04:38:43Z,1876245,0.425,0.555,0.020,797404.125,1041315.975,37524.900,-5968.011,-1.372180e-01,-107662.642,-1.189555e-01,-81215.347,-7.235018e-02,-194846
286,2020-11-09T21:36:09Z,3270529,0.454,0.525,0.021,1484820.166,1717027.725,68681.109,-16.800,-2.445489e-04,-363.200,-2.445489e-04,-420.000,-2.445489e-04,-800
289,2020-11-09T21:57:29Z,3270531,0.454,0.525,0.021,1484821.074,1717028.775,68681.151,-0.021,-3.057607e-07,-0.454,-3.057607e-07,-0.525,-3.057607e-07,-1
290,2020-11-09T22:04:38Z,3270530,0.454,0.525,0.021,1484820.620,1717028.250,68681.130,-0.021,-3.057607e-07,-0.454,-3.057607e-07,-0.525,-3.057607e-07,-1
292,2020-11-09T22:17:34Z,3270558,0.454,0.525,0.021,1484833.332,1717042.950,68681.718,-0.042,-6.115161e-07,-0.908,-6.115161e-07,-1.050,-6.115161e-07,-2


In [117]:
#Woah, why's that? 
df[df[' biden change'] < 0]

,timestamp,total votes,trump %,biden %,3rd party %,trump votes,biden votes,3rd party votes,3rd party change,3rd party % change,trump change,trump % change,biden change,biden % change,total change
29,2020-11-04T04:06:06Z,1511429,0.394,0.576,0.030,595503.026,870583.104,45342.870,1532.773,3.498675e-02,289.984,4.871936e-04,-1086.757,-1.246753e-03,736
33,2020-11-04T04:14:11Z,1583375,0.407,0.573,0.020,644433.625,907273.875,31667.500,24.640,7.786907e-04,2083.567,3.243663e-03,-876.207,-9.648262e-04,1232
38,2020-11-04T04:24:09Z,1830835,0.421,0.558,0.021,770781.535,1021605.930,38447.535,1871.855,5.117759e-02,863.471,1.121510e-03,-684.326,-6.694048e-04,2051
41,2020-11-04T04:31:23Z,1836796,0.421,0.558,0.021,773291.116,1024932.168,38572.716,1839.776,5.008518e-02,62.729,8.112610e-05,-1753.505,-1.707928e-03,149
45,2020-11-04T04:34:57Z,2045867,0.435,0.544,0.021,889952.145,1112951.648,42963.207,64.974,1.514608e-03,3388.663,3.822245e-03,-359.637,-3.230336e-04,3094
49,2020-11-04T04:37:38Z,2069583,0.437,0.542,0.021,904407.771,1121713.986,43461.243,66.801,1.539391e-03,3456.499,3.836499e-03,-342.300,-3.050649e-04,3181
51,2020-11-04T04:38:43Z,1876245,0.425,0.555,0.020,797404.125,1041315.975,37524.900,-5968.011,-1.372180e-01,-107662.642,-1.189555e-01,-81215.347,-7.235018e-02,-194846
155,2020-11-04T16:18:51Z,3202507,0.454,0.525,0.021,1453938.178,1681316.175,67252.647,3212.847,5.016954e-02,234.718,1.614621e-04,-2930.565,-1.739986e-03,517
178,2020-11-04T22:18:49Z,3227153,0.454,0.525,0.021,1465127.462,1694255.325,67770.213,3293.613,5.108230e-02,1508.642,1.030762e-03,-1479.255,-8.723388e-04,3323
286,2020-11-09T21:36:09Z,3270529,0.454,0.525,0.021,1484820.166,1717027.725,68681.109,-16.800,-2.445489e-04,-363.200,-2.445489e-04,-420.000,-2.445489e-04,-800


In [130]:
print(np.max(np.abs(df[df['3rd party change'] < 0]['3rd party change'])/df['total votes']))

0.008901517204813731


In [145]:
np.sum(np.abs(df[df['3rd party change'] < 0]['3rd party change'])/df['total votes'] > .003)

2

In [144]:
np.sum(np.abs(df[df[' biden change'] < 0]['3rd party change'])/df['total votes'] > .002)

1

In [148]:
.002 * np.max(df['total votes'])

6544.928